In [4]:
%matplotlib inline


Autograd: 自动求导机制
===================================

PyTorch 中所有神经网络的核心是 ``autograd`` 包。
我们先简单介绍一下这个包，然后训练第一个简单的神经网络。

``autograd``包为张量上的所有操作提供了自动求导。
它是一个在运行时定义的框架，这意味着反向传播是根据你的代码来确定如何运行，并且每次迭代可以是不同的。


示例

张量（Tensor）
--------

``torch.Tensor``是这个包的核心类。如果设置
``.requires_grad`` 为 ``True``，那么将会追踪所有对于该张量的操作。 
当完成计算后通过调用 ``.backward()``，自动计算所有的梯度，
这个张量的所有梯度将会自动积累到 ``.grad`` 属性。

要阻止张量跟踪历史记录，可以调用``.detach()``方法将其与计算历史记录分离，并禁止跟踪它将来的计算记录。

为了防止跟踪历史记录（和使用内存），可以将代码块包装在``with torch.no_grad()：``中。
在评估模型时特别有用，因为模型可能具有`requires_grad = True`的可训练参数，但是我们不需要梯度计算。

在自动梯度计算中还有另外一个重要的类``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

``Tensor`` 和 ``Function``互相连接并生成一个非循环图，它表示和存储了完整的计算历史。
每个张量都有一个``.grad_fn``属性，这个属性引用了一个创建了``Tensor``的``Function``（除非这个张量是用户手动创建的，即，这个张量的
``grad_fn`` 是 ``None``）。

如果需要计算导数，你可以在``Tensor``上调用``.backward()``。 
如果``Tensor``是一个标量（即它包含一个元素数据）则不需要为``backward()``指定任何参数，
但是如果它有更多的元素，你需要指定一个``gradient`` 参数来匹配张量的形状。


***译者注：在其他的文章中你可能会看到说将Tensor包裹到Variable中提供自动梯度计算，Variable 这个在0.41版中已经被标注为过期了，现在可以直接使用Tensor，官方文档在这里：***
(https://pytorch.org/docs/stable/autograd.html#variable-deprecated) 

具体的后面会有详细说明

In [2]:
import torch

In [4]:
#创建一个张量并设置 requires_grad=True 用来追踪他的计算历史
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [5]:
#对张量进行操作:
y = x + 2
print(y)
#结果y已经被计算出来了，所以，grad_fn已经被自动生成了。

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [6]:
print(y.grad_fn)

In [7]:
#对y进行一个操作
z = y * y * 3

out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward1>)


In [12]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
#.requires_grad_( ... ) 可以改变现有张量的 requires_grad属性。
# 如果没有指定的话，默认输入的flag是 False。
print(a.requires_grad)  # False 默认
a.requires_grad_(True)
print(a.requires_grad) # True
print("a")
print(a)
print((a * a))
b = (a * a).sum()  # a中元素的平方和
print("b")
print(b)
print(b.grad_fn)

False
True
a
tensor([[ 2.0697, -0.5628],
        [ 6.5453,  0.5896]], requires_grad=True)
tensor([[ 4.2835,  0.3167],
        [42.8409,  0.3476]], grad_fn=<MulBackward0>)
b
tensor(47.7887, grad_fn=<SumBackward0>)


梯度
---------
反向传播
因为 ``out``是一个纯量（scalar），``out.backward()`` 等于``out.backward(torch.tensor(1))``。



In [13]:
#反向传播 因为 out是一个纯量（scalar），out.backward() 等于out.backward(torch.tensor(1))。
out.backward()

print gradients d(out)/dx




In [12]:
print(x.grad)  # d(out)/dx

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


得到矩阵 ``4.5``.调用 ``out``
*Tensor* “$o$”.

x=tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
        
y=x+2   

z=y * y * 3

out=z.mean()

得到 $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

因此,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



可以使用 autograd 做更多的操作



In [25]:
x = torch.randn(3, requires_grad=True)
print("x")
print(x)

y = x * 2
print("y")
print(y)
while y.data.norm() < 1000:
    print("y.data.norm()")
    print(y.data.norm())
    y = y * 2
    print("y")
    print(y)
    print("y/x")
    print(y/x)
print("y final")
print(y)

x
tensor([ 1.4972, -0.9856, -0.3498], requires_grad=True)
y
tensor([ 2.9944, -1.9712, -0.6996], grad_fn=<MulBackward0>)
y.data.norm()
tensor(3.6526)
y
tensor([ 5.9888, -3.9423, -1.3992], grad_fn=<MulBackward0>)
y/x
tensor([4., 4., 4.], grad_fn=<DivBackward0>)
y.data.norm()
tensor(7.3052)
y
tensor([11.9776, -7.8846, -2.7985], grad_fn=<MulBackward0>)
y/x
tensor([8., 8., 8.], grad_fn=<DivBackward0>)
y.data.norm()
tensor(14.6103)
y
tensor([ 23.9552, -15.7692,  -5.5969], grad_fn=<MulBackward0>)
y/x
tensor([16., 16., 16.], grad_fn=<DivBackward0>)
y.data.norm()
tensor(29.2206)
y
tensor([ 47.9104, -31.5384, -11.1938], grad_fn=<MulBackward0>)
y/x
tensor([32., 32., 32.], grad_fn=<DivBackward0>)
y.data.norm()
tensor(58.4412)
y
tensor([ 95.8208, -63.0768, -22.3876], grad_fn=<MulBackward0>)
y/x
tensor([64., 64., 64.], grad_fn=<DivBackward0>)
y.data.norm()
tensor(116.8825)
y
tensor([ 191.6415, -126.1537,  -44.7753], grad_fn=<MulBackward0>)
y/x
tensor([128., 128., 128.], grad_fn=<DivBackward0>)
y.dat

In [21]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)

print(gradients.size())
y.backward(gradients)

print(x.grad)
'''  
torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])
分析：
中间1.0  正常算出来的梯度 
前面0.1  正常算出来的梯度 *0.1
'''

torch.Size([3])
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


如果``.requires_grad=True``但是你又不希望进行autograd的计算，
那么可以将变量包裹在 ``with torch.no_grad()``中:


In [29]:
print(x)
print(x.requires_grad)
print((x ** 2))  # x的平方  tensor([2.2416, 0.9714, 0.1224], grad_fn=<PowBackward0>)
print((x * x))  # x的平方 tensor([2.2416, 0.9714, 0.1224], grad_fn=<MulBackward0>)
print((x * 2))  # x*2 tensor([ 2.9944, -1.9712, -0.6996], grad_fn=<MulBackward0>)
print((x ** 2).requires_grad)


print((x ** 3))  # x的3次方  tensor([2.2416, 0.9714, 0.1224], grad_fn=<PowBackward0>)
print((x * x *x))  # x的3次方 tensor([2.2416, 0.9714, 0.1224], grad_fn=<MulBackward0>)


with torch.no_grad():
	print((x ** 2).requires_grad)

tensor([ 1.4972, -0.9856, -0.3498], requires_grad=True)
True
tensor([2.2416, 0.9714, 0.1224], grad_fn=<PowBackward0>)
tensor([2.2416, 0.9714, 0.1224], grad_fn=<MulBackward0>)
tensor([ 2.9944, -1.9712, -0.6996], grad_fn=<MulBackward0>)
True
tensor([ 3.3561, -0.9573, -0.0428], grad_fn=<PowBackward0>)
tensor([ 3.3561, -0.9573, -0.0428], grad_fn=<MulBackward0>)
False


**稍后阅读:**

 ``autograd`` 和 ``Function`` 的官方文档 https://pytorch.org/docs/autograd

